# ***Installing missing libraries***

In [2]:
pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 111.0 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install --upgrade gupload

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 468.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 907.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 436.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 541.4 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
celery 5.2.7 requires click<9.0,>=8.0.3, but you have click 7.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
pip install google.colab

In [4]:
pip install mystic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.8/391.8 kB 961.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 510.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 608.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 1.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 970.8 kB/s eta 0:00:0000:0100:01


# ***Libraries***

In [ ]:
import numpy as np                                                                # used to create tables type numpy
import matplotlib.pyplot as plt
import random as rn                                                               # randomness
import math                                                                       # math library
from mystic.symbolic import generate_penalty, generate_conditions                 # might use is later to generate database using genetic algorithm // view lin https://stackoverflow.com/questions/57222616/find-function-minimum-for-discrete-variables-w-bounds-and-constraints
from scipy.stats import invgauss                                                  # advanced calculus
import matplotlib.pyplot as plt                                                   # plotting results
import xlsxwriter                                                                 # generate excel file
from openpyxl import Workbook                                                     # needed to generate excel file
from matplotlib import cm                                                         # used to change graph's colors
from google.colab import drive                                                    # linking the code with the drive
from pydrive.auth import GoogleAuth                                               # Authentificate the connection between code and drive    
from google.colab import auth
import time                                                                       # Time library
import itertools                                                                  # used to make all permutations and combinations of a set

In [ ]:
from google.colab import drive                            #coonnecting drive to current tab
drive.mount('/content/gdrive')   

# ***Useful functions***

In [1]:
def distance(x1, y1,x2 ,y2):                                # will be used to calculate distance between sectors to generate Qij
    return ((x1-x2)**2+(y1-y2)**2)**0.5
def flatten(input):                                         # used to convert 2D matrix into 1D vector   [[1,2],[3,4]] ====> [1, 2, 3, 4]
    new_list = []
    for i in input:
        for j in i:
            new_list.append(j)
    return new_list
def generateQs(sects):                                      # generate a list of Qijs of each sector
    Qs=[]
    for i in sects:
        temp=[]
        for j in sects:
            if i.ident==j.ident:                            # cost of distribution from site x to site x =0
                temp.append(0)
            else:
                Qij=i.getQij(j)                             # if site x different from site y the function/method .getQij(site) calculates the cost of transportation based on the distance between the two sites
                temp.append(Qij)
        Qs.append(temp)
    #print(Qs)
    return Qs

def InventoryMain(ProdPlan, InvPlan, sigma, Sectors, Mp, Mc, alpha, beta, lambda0, dt, Qmax):
  Cor=[[] for i in range(len(ProdPlan))]
  for j in range(len(Sectors)):
    for i in range(len(InvPlan)):
      Cor[i].append(sigma*(ProdPlan[i][j]+InvPlan[i][j]))
  BestN=Bestmaintenance1(Cor,Sectors,Mp,Mc,alpha,beta,lambda0,dt,Qmax)
  return BestN
# Maintenance plan of storage tank
def Bestmaintenancetank(ProductionPlan,Sectors,Mp,Mc,alpha,beta,lambda0,dt,Qmax):        # generate best maintenance plan
  H = len(ProductionPlan)
  nbSectors = len(ProductionPlan[0])
  BestNs=[]                                         # varaible created to save the best results
  Mctot=[]                                          # variables contrains the costs
  History=[]  
  z=0.3                                      #this function will calculate the area of lamda using triangles and rectangles
  x1=math.exp(-math.exp(-((z-beta)/alpha)))                   #calculate the surface created by lambda0
  for j in range(nbSectors):                             # for each sector the planification is calculated on its own
    tempc=[]                                             # this variable will cumulate the area between two preventive maintenances
    for k in range(H-1):
      s,temp,last=0,0,0                                   # s will store the overall area value
      counter = 1
      for i in range(H):
        if (i*(k+2)/(H)<= counter and (i+1)*(k+2)/H>counter and counter<=k+1):              # counter is the index of the preventive intervention
        #if (i%(H//(k+2))==0 and i!=0):                                                       the objective of this condition is to locate periods of peventive maintenances in order to reset lambda to lambda0
          counter+=1
          s+=temp
          temp=((ProductionPlan[i-1][j]/Qmax)*alpha*dt/(beta**2))/2                # initiate temp to the area of the triangle
          last=0
          #print(f"Pmaintenance in Period = {i} for N* = {k+1}")
        else:
          last+=((ProductionPlan[i-1][j]/Qmax)*alpha*dt/(beta**2))/2               # calculate the area of lambda at period t
          temp+=last                                                                           # add the area calculated to temp
          last+=((ProductionPlan[i-1][j]/Qmax)*alpha*dt/(beta**2))/2                 # the area of the triangle becomes a rectangle in the next period
      tempc.append(Mp*(k+1)+Mc*(s+x0+temp))                                                       # calculate the cost of maintenance for each Nmp
    x=min(tempc)                                                                                    # determine the minumum cost of all Ns 
    N=tempc.index(x)                                                                                  # locate the plan with the least maintenance cost
    BestNs.append((x,N))                                                                                # add the best result of site j to the overall best result
    History.append(tempc)
  return BestNs
# Maintenance plan of machine
def Bestmaintenance(ProductionPlan,Sectors,Mp,Mc,alpha,beta,lambda0,dt):        # generate best maintenance plan of machine
  H = len(ProductionPlan)                                 
  nbSectors = len(ProductionPlan[0])
  BestNs=[]                                         # varaible created to save the best results
  Mctot=[]                                          # variables contrains the costs
  History=[]                                        #this function will calculate the area of lamda using triangles and rectangles
  x0=lambda0*dt*H*alpha/(beta**2)                   #calculate the surface created by lambda0
  for j in range(nbSectors):                             # for each sector the planification is calculated on its own
    tempc=[]                                             # this variable will cumulate the area between two preventive maintenances
    for k in range(H-1):
      s,temp,last=0,0,0                                   # s will store the overall area value
      counter = 1
      for i in range(H):
        if (i*(k+2)/(H)<= counter and (i+1)*(k+2)/H>counter and counter<=k+1):              # counter is the index of the preventive intervention
        #if (i%(H//(k+2))==0 and i!=0):                                                       the objective of this condition is to locate periods of peventive maintenances in order to reset lambda to lambda0
          counter+=1
          s+=temp
          temp=((ProductionPlan[i-1][j]/Sectors[j].Umax)*alpha*dt/(beta**2))/2                # initiate temp to the area of the triangle
          last=0
          #print(f"Pmaintenance in Period = {i} for N* = {k+1}")
        else:
          last+=((ProductionPlan[i-1][j]/Sectors[j].Umax)*alpha*dt/(beta**2))/2               # calculate the area of lambda at period t
          temp+=last                                                                           # add the area calculated to temp
          last+=((ProductionPlan[i-1][j]/Sectors[j].Umax)*alpha*dt/(beta**2))/2                 # the area of the triangle becomes a rectangle in the next period
      tempc.append(Mp*(k+1)+Mc*(s+x0+temp))                                                       # calculate the cost of maintenance for each Nmp
    x=min(tempc)                                                                                    # determine the minumum cost of all Ns 
    N=tempc.index(x)                                                                                  # locate the plan with the least maintenance cost
    BestNs.append((x,N))                                                                                # add the best result of site j to the overall best result
    History.append(tempc)
  print(BestNs)
  return BestNs

def SubsPos(L):
    Subs = []
    for i in range(1, len(L)+1):
        for subset in itertools.permutations(L, i):                                                         #generate all subsets of donors
            Subs.append(subset)
    return(Subs)
def SubsNeg(L):
  a=itertools.permutations(L)                                                                               # genetate all permutations of sites in defecit
  return list(a)

def BestTrans(Stock,Donors,Recievers,Sectors,nbSectors):
  InitialStock=Stock.copy()
  DonorsComb = SubsPos(Donors)
  RecieversComb = SubsNeg(Recievers)
  AllCombs=[]
  HighProf = 20**10
  Best = [[0 for l in range(nbSectors)] for i in range(nbSectors)]
  for i in DonorsComb:
    for k in RecieversComb:                                                                   #initiate the combination of donors/recievers to evaluate
      TransList = [[0 for l in range(nbSectors)] for i in range(nbSectors)]
      TempSt=InitialStock.copy()                                                              # make a copy of initial stock to check the scenareo
      profit = 0                                                                              # initialize profit
      for j in i:
        for h in k:
          if len(i)*len(k)!=0:                                                                # if both sets not empty
            if TempSt[j]>0 and TempSt[h]<0:                                                   # verify that the donor still can donate
              excess = TempSt[j]                                                              
              defecit = abs(TempSt[h])                                                        
              transportable = min(excess, defecit)                                            # the maximum quantity that can be transported to satisfy the shortage
              transportableMin = Sectors[j].v[h] * (transportable //
                                                              Sectors[j].v[h])                # deduct the last vehicle from transportable
              transportablePlus = transportable + rn.randint(
                          0, Sectors[j].v[h] -
                          (math.floor(transportable) % math.floor(Sectors[j].v[h])))          # add products to the last vehicle
              profit1 = Sectors[j].cs * transportablePlus + Sectors[
                          h].cl * min(transportablePlus,defecit) - Sectors[j].getQij(
                              Sectors[h]) * (
                                  (transportablePlus // Sectors[j].v[h]) + 1)-Sectors[h].cs*max(0,transportablePlus-defecit)     # calculate profit of transportable min
              profit2 = Sectors[j].cs * transportableMin + Sectors[
                          h].cl * min(transportableMin,defecit) - Sectors[j].getQij(
                              Sectors[h]) * (transportableMin // Sectors[j].v[h])-Sectors[h].cs*max(0,transportablePlus-defecit) # calculate profit of transportable max
              if profit1<0 and profit2<0:
                TransList[j][h]=0
              elif (profit1 > profit2):
                TransList[j][h]=transportablePlus
                profit+=profit1
              else:
                TransList[j][h]=transportableMin
                profit+=profit1
      AllCombs.append([profit,i,k,TransList])             # save the results of the combination calculated
  for i in AllCombs:
    if HighProf>i[0]:
      Best=i[-1]
      HighProf=i[0]
  return Best

def BestResultWriter(param,ProdPlan,StorPlan,TransPlan,DemandFore,id):
  nbSectors, H, Sectors = param[0], param[1], param[2]
  ExcelFile = f'/content/gdrive/MyDrive/QualitaResults/ResultsAnalysysNew{id}.xlsx'
  prodWorkBook = xlsxwriter.Workbook(ExcelFile)
  PWS=prodWorkBook.add_worksheet()
  PWS.write(0,0,nbSectors)
  PWS.write(0,1,H)

  for i in range(H):
    for j in range(nbSectors):
      if i==0:
        PWS.write(0,j,f"DemandSector{j+1}")
      PWS.write(i+1,j,DemandFore[j][i])

  decal = nbSectors*1+3

  for i in range(H):
    for j in range(nbSectors):
      if i==0:
        PWS.write(0,decal+j,f"ProductionSector{j+1}")
      PWS.write(i+1,decal+j,ProdPlan[i][j])

  decal1=nbSectors*2+6

  for i in range(H):
    for j in range(nbSectors):
      if i==0:
        PWS.write(0,decal1+j,f"StorageSector{j+1}")
      PWS.write(i+1,decal1+j,StorPlan[i][j])
  
  decal2=nbSectors*3+9

  for i in range(H):
    for j in range(nbSectors):
      for k in range(nbSectors):
        if i==0:
          PWS.write(0,decal2+j*nbSectors+k,f"Transaction from {j+1} to {k+1}")
        PWS.write(i+1,decal2+j*nbSectors+k,TransPlan[i][j][k])
  prodWorkBook.close()

def BestResultWriterEver(param,ExtraInf,ManufacturingData,History,id):
  ProdPlan,StorPlan,TransPlan,DemandFore=ManufacturingData[0],ManufacturingData[1],ManufacturingData[2],ManufacturingData[3]
  TimeFrame,Generation,BestFitness,FinessDiff,MaintenanceCost=History[0],History[1],History[2],History[3],History[4]
  nbSectors, H, Sectors = param[0], param[1], param[2]
  GAStats,MaintenanceInf,MaintenanceData = ExtraInf[0],ExtraInf[1],ExtraInf[2]
  ExcelFile = f'/content/gdrive/MyDrive/QualitaResults/BestWriterRandomProdMaint{id}.xlsx'
  prodWorkBook = xlsxwriter.Workbook(ExcelFile)
  PWS=prodWorkBook.add_worksheet()
  PWS.write(0,0,f"Number of sites = {nbSectors}")
  PWS.write(0,1,f"Number of periods = {H}")
  PWS.write(0,2,f"Population Size = {GAStats[0]}")
  PWS.write(0,3,f"Mutation rate = {GAStats[1]}")
  PWS.write(0,4,f"Crossover rate = {GAStats[2]}")
  PWS.write(0,5,f"Preentive maintenace = {MaintenanceInf[0]}")
  PWS.write(0,6,f"Corrective maintenace = {MaintenanceInf[1]}")
  PWS.write(0,7,f"Alpha = {MaintenanceInf[2]}")
  PWS.write(0,8,f"Beta = {MaintenanceInf[3]}")
  PWS.write(0,9,f"lambda0 = {MaintenanceInf[4]}")

  for j in range(nbSectors):
    PWS.write(3,j+1,f"Site {j+1}")
  PWS.write(4,0,f"Manufacturing Cost")
  PWS.write(5,0,f"Storage Cost")
  PWS.write(6,0,f"Shortage Cost ")
  PWS.write(7,0,f"Shipping Cost")
  PWS.write(8,0,f"Vehicle Capacity")
  PWS.write(9,0,f"Maximum Manufacturing rate")
  PWS.write(10,0,f"Minimum Manufacturing rate")

  for j in range(nbSectors):
    PWS.write(4,j+1,Sectors[j].cp)
    PWS.write(5,j+1,Sectors[j].cs)
    PWS.write(6,j+1,Sectors[j].cl)
    PWS.write(7,j+1,str(Sectors[j].Qijs).strip('[]'))
    PWS.write(8,j+1,str(Sectors[j].v).strip('[]'))
    PWS.write(9,j+1,Sectors[j].Umax)
    PWS.write(10,j+1,Sectors[j].Umin)

  Spacing=12
  for i in range(H):
    for j in range(nbSectors):
      if i==0:
        PWS.write(Spacing,j,f"DemandSector{j+1}")
      PWS.write(i+1+Spacing,j,DemandFore[j][i])

  decal = nbSectors*1+3
  for i in range(H):
    for j in range(nbSectors):
      if i==0:
        PWS.write(Spacing,decal+j,f"ProductionSector{j+1}")
      PWS.write(i+1+Spacing,decal+j,ProdPlan[i][j])

  decal1=nbSectors*2+6

  for i in range(H):
    for j in range(nbSectors):
      if i==0:
        PWS.write(Spacing,decal1+j,f"StorageSector{j+1}")
      PWS.write(i+1+Spacing,decal1+j,StorPlan[i][j])
  
  decal2=nbSectors*3+9

  for i in range(H):
    for j in range(nbSectors):
      for k in range(nbSectors):
        if i==0:
          PWS.write(Spacing,decal2+j*nbSectors+k,f"Transaction from {j+1} to {k+1}")
        PWS.write(i+1+Spacing,decal2+j*nbSectors+k,TransPlan[i][j][k])

  decal3 = nbSectors*4+12
  for i in range(nbSectors):
    if i==0:
      PWS.write(Spacing-7,decal3+j*nbSectors+k,f"Maintenance site {i+1}")
      PWS.write(Spacing+1-7,decal2+j*nbSectors-1,"Maintenance Cost")
      PWS.write(Spacing+2-7,decal2+j*nbSectors-1,"Maintenance number")
    PWS.write(Spacing+1-7,decal2+j*nbSectors+i,MaintenanceData[i][0])
    PWS.write(Spacing+2-7,decal2+j*nbSectors+i,MaintenanceData[i][1])

  Spacing2 = Spacing+H+5

  PWS.write(Spacing2,0,"Time")
  PWS.write(Spacing2,1,"Generation")
  PWS.write(Spacing2,2,"Best Cost")
  PWS.write(Spacing2,3,"Maintenance Cost")
  PWS.write(Spacing2,4,"Difference Best Worst genes")
  for i in range(len(TimeFrame)):
    PWS.write(Spacing2+i+1,0,TimeFrame[i])
    PWS.write(Spacing2+i+1,1,Generation[i])
    PWS.write(Spacing2+i+1,2,BestFitness[i])
    PWS.write(Spacing2+i+1,3,MaintenanceCost[i])
    PWS.write(Spacing2+i+1,4,FinessDiff[i])
  prodWorkBook.close()

def ProdCost(ProdPlan,StorPlan,ShippingPlan,MaintenanceList,param):
  nbSectors, H, Sectors = param[0],param[1],param[2]
  print(f"ProdPlan={ProdPlan}/nStorPlan={StorPlan}/nShippingPlan={ShippingPlan}/nMaintenancePlan={MaintenanceList}")
  totalCost=0
  for t in range(1,H):
    for i in range(nbSectors):
      totalCost+=Sectors[i].cp * ProdPlan[t][i] + Sectors[
                  i].cl * abs(StorPlan[t][i]) * Sectors[i].checkShort(StorPlan[t][i]) + Sectors[
                      i].cs * abs(StorPlan[t][i]) * Sectors[i].checkStock(StorPlan[t][i])
      for j in range(nbSectors):
        if j!=i or ShippingPlan[t][i][j]:
          totalCost+=Sectors[i].getQij(
                      Sectors[j]) * (ShippingPlan[t][i][j]//Sectors[i].v[j])
  for i in MaintenanceList:
    totalCost+=i[0]
  return totalCost

# ***Sector class***

In [ ]:
class Sector:
    def __init__(self, ident, x, y, cp, cl, cs, v, Umax, Umin, fuelCost):
        self.ident, self.x, self.y, self.cp, self.cl, self.cs, self.v, self.cs, self.Umax, self.Umin, self.fuelCost = ident, x, y, cp, cl, cs, v, cs, Umax, Umin, fuelCost
        self.Qijs = []
        self.indicator = True                                     #indicate deficit 
        self.blocked = False                                      #indicate whether a sector is able to give units/transport outwards (used to avoid giving and taking at the same time)

    def getPos(self):                                             #used to generate Qijs at a later stage
        return (self.x, self.y)

    def checkStock(self,x):                                       #emulate Alphaij
      if x>=0:
        return 1
      else:
        return 0
    
    def checkShort(self,x):                                       #emulate Alphaij
      if x<0:
        return 1
      else:
        return 0
      
    def setCP(self, cp):                                          #Setters and getters
        self.cp = cp

    def setCL(self, cp):
        self.cl = cl

    def setStock(self, stock):
        self.s = stock

    def getQij(self, sect):                                                             #calculate the cost of transportation from the curent sector to another sector
        (x2, y2) = sect.getPos()
        return self.fuelCost * distance(self.x, x2, self.y, y2)                         #Qijs are based on real data (fuel cost and distance)

    def prodCost(self, Uit):
        return Uit * self.cp

    def QijList(self, Qs):                                                              #save the transportation costs
        self.Qijs = Qs

    def showInf(self):                                                                  #Debug & show informations
        print(
            f"Sector ID = {self.ident}\nSector Position (x,y) : ({self.x},{self.y})\nProduction cost : {self.cp:.2f} euro/unit.period\nShortage penality cost : {self.cl:.2f} euro/unit\nStorage cost : {self.cs:.2f} euro/unit\nThe production rate varies between {self.Umin:.2f} and {self.Umax:.2f} units/period"
        )
        for i in range(len(self.v)):
            print(
                f"The Vehicle capacity from sector {self.ident} to sector {i} : {self.v[i]} units/vehicle"
            )
        if (len(self.Qijs) == 0):
            print(
                f"Please set the cost of vehicles from sector {self.ident} to each sector"
            )
        else:
            print(f"The fuel consumption cost is {self.fuelCost:.2f} euro/km")
            for i in range(len(self.Qijs)):
                print(
                    f"The transportation cost from sector {self.ident} to sector {i} : {self.Qijs[i]:.2f} euro/vehicle"
                )
        print("#"*30)

# ***Generate demand & Sectors***

In [ ]:
dt = 1                                              #period (didnt implement it yet, just change the the demand amplitude of the number of periods H = H/dt)
H = 10
nbSectors = 3
maxDemand = 500
minDemand = 50
serviceLevel = 0.9

Dem_Dist_Func = 1  #1-Gaussian distribution     2-Sinusoidal distribution       3-Quasi-constant demand
number_of_periods = 2*H/12  #for the sinusoidal demand distribution
noise_amp = 150  #introduce noise to signals

DemandList = []

n = H * nbSectors  #Population
mu, sigma = (minDemand + maxDemand) / 2, 90  # mean and standard deviation
DemDis = np.random.normal(mu, sigma, n)

for i in range(nbSectors):                            #generate Demand list based on the chosen distribtion function
    temp = []
    for j in range(H):
        if (Dem_Dist_Func == 1):
            temp.append(math.floor(DemDis[i * nbSectors + j]))
        elif (Dem_Dist_Func == 2):
            temp.append((3 * minDemand + maxDemand) / 2 +
                        noise_amp * rn.random() + (maxDemand - minDemand) *
                        math.sin(2 * math.pi * number_of_periods *
                                 (i * nbSectors + j) / (H * nbSectors)) / 2)
        elif (Dem_Dist_Func == 3):
            temp.append(
                ((maxDemand + minDemand) + noise_amp * rn.random()) / 2)
    DemandList.append(temp)
minCp, maxCp = 1, 4
minCs, maxCs = 10, 14
minCl, maxCl = 20, 24
maxX, maxY = 1000, 1000
minVCap, maxVCap = 30, 70
minMinProd, maxMinProd, minMaxProd, maxMaxProd = 0, 250, 300, 600
fuelCost = 1.877       #Fuel cost : euro/liter
minFuelPerKm, maxFuelPerKm = 0.08,0.12       #fuel consumption : litre/km
minFuelCost, maxFuelCost = fuelCost*minFuelPerKm , fuelCost*maxFuelPerKm
Sectors = []
for i in range(nbSectors):
    cp = rn.uniform(minCp, maxCp)
    cs = rn.uniform(minCs, maxCs)
    cl = serviceLevel * rn.uniform(minCl, maxCl)
    x = rn.randint(0, maxX)
    y = rn.randint(0, maxY)
    fuelCost = rn.uniform(minFuelCost,maxFuelCost)
    Vij = []
    for j in range(nbSectors):
        if i == j:
            Vij.append(0)
        else:
            Vij.append(rn.randint(minVCap, maxVCap))
    Umax, Umin = rn.randint(minMaxProd,
                            maxMaxProd), rn.randint(minMinProd, maxMinProd)
    sect = Sector(i, x, y, cp, cl, cs, Vij, Umax, Umin, fuelCost)
    Sectors.append(sect)
QsList = generateQs(Sectors)
for i in range(nbSectors):
    Sectors[i].QijList(QsList[i])
    Sectors[i].showInf()

Sector ID = 0
Sector Position (x,y) : (186,227)
Production cost : 2.30 euro/unit.period
Shortage penality cost : 21.36 euro/unit
Storage cost : 11.87 euro/unit
The production rate varies between 86.00 and 523.00 units/period
The Vehicle capacity from sector 0 to sector 0 : 0 units/vehicle
The Vehicle capacity from sector 0 to sector 1 : 38 units/vehicle
The Vehicle capacity from sector 0 to sector 2 : 33 units/vehicle
The fuel consumption cost is 0.20 euro/km
The transportation cost from sector 0 to sector 0 : 0.00 euro/vehicle
The transportation cost from sector 0 to sector 1 : 41.24 euro/vehicle
The transportation cost from sector 0 to sector 2 : 54.01 euro/vehicle
##############################
Sector ID = 1
Sector Position (x,y) : (232,438)
Production cost : 1.07 euro/unit.period
Shortage penality cost : 18.58 euro/unit
Storage cost : 13.10 euro/unit
The production rate varies between 216.00 and 569.00 units/period
The Vehicle capacity from sector 1 to sector 0 : 33 units/vehicle
T

# ***Estimate number of iterations***

In [ ]:
import math
maxAvProd=450
minAvProd=50
vehAvCap=100
toler=1
nbSectors=3
H=10
Univers = ((maxAvProd-minAvProd)/(vehAvCap/toler))**(nbSectors*H)
p=1/Univers
q=1-p
solProb = 0.2
minN = math.log(1-solProb)/math.log(q)
print(minN)

ZeroDivisionError: ignored

# ***Saving Configuration***

In [ ]:
id = 3
param=[nbSectors, H, Sectors]#/content/gdrive/MyDrive/PFEDATA
np.save(f"/content/gdrive/MyDrive/CommonPlace/Situation{id}.npy",[param,DemandList],allow_pickle=True)

FileNotFoundError: ignored

# ***Reload Data***

In [ ]:
id=3
Reload=np.load(f"/content/gdrive/MyDrive/CommonPlace/Situation{id}.npy",allow_pickle=True)
param,DemandList = Reload[0],Reload[1]
nbSectors, H, Sectors = param[0],param[1],param[2]
DemandList

# ***Random solver***

In [ ]:
ExcelFile = f'/content/gdrive/MyDrive/QualitaResults/ResultsAnalysysNew{id}.xlsx'
#ExcelFile = f'/content/gdrive/MyDrive/phd/ResultsAnalysysNew{id}.xlsx'
nbSectors, H, Sectors = param[0],param[1],param[2]
prodWorkBook = xlsxwriter.Workbook(ExcelFile)
PWS=prodWorkBook.add_worksheet()
PWS.write(0,0,nbSectors)
PWS.write(0,1,H)
row = 1
Mp, Mc= 100,7000
Mp1, Mc1= 10000,7000000
alpha, beta = 2,20
lambda0=0.01
sigma=0.1
Qmax=1000

param=[nbSectors,H,Sectors]
ExtraInf=[[0,0,0],[Mp,Mc,alpha,beta,lambda0],[]]
History = [[],[],[],[],[]]

saved = [DemandList]
savedalt = [DemandList]
MaintenanceImpact =0.7
Mincost = 10**10
Mincost1 = 10**10
dt=1
last=0
notFirstTime=False
BestProdPlan=[]
BaseTime = time.time()
for k in range(300):
  if (k%5==0):
    print(k)
  test = True     
  MinCostProd=10**10
  for db in range(500):      #Number of iterations to improve the production plan
    validate = True
    if db%10000==0:
      print(f"Generation {k}, Iteration {db}")
    ProdList = [[0 for i in range(nbSectors)] for j in range(H)]
    StorList = [[ 0  for i in range(nbSectors)] for j in range(H)]
    TransList = [[[0 for l in range(nbSectors)] for i in range(nbSectors)] for j in range(H)]
    CurrentCost = 0
    for t in range(H):    #Generate Production plan for iteratio db 
      counter =1                # counting the number of preventive interventions
      for p in range(nbSectors):
        if notFirstTime:          #meaning a maintenance plan already exists and the production plan is adjusted to the maintenance periods
          if t*(BestSol[p][1]+1)/(H)<= counter and (t+1)*(BestSol[p][1]+1)/H>counter:
            x = math.floor((rn.randint(0,Sectors[p].Umax - Sectors[p].Umin) + Sectors[p].Umin)*MaintenanceImpact)
            counter+=1
          else :
            x = math.floor(rn.randint(0,Sectors[p].Umax - Sectors[p].Umin) + Sectors[p].Umin)
        else:
          x = rn.randint(0,Sectors[p].Umax - Sectors[p].Umin) + Sectors[p].Umin
        ProdList[t][p]=x
    for p in range(nbSectors):    #Initiate Storage
      StorList[0][p]=ProdList[0][p]
    for t in range(1, H):         #loop throught periods H 
      Donors,Recievers=[],[]
      for p in range(nbSectors):                            #initiate indicators/blockers
        if (ProdList[t][p] + StorList[t-1][p] - DemandList[p][t] > 0):
          Sectors[p].indicator, Sectors[p].blocked = False, False
          Donors.append(p)
        else:
          Sectors[p].indicator, Sectors[p].blocked = True, True      #indicate deficit      #block outward transactions
          Recievers.append(p)
      for v in range(nbSectors):          #calculate the storage of each sector at the beginning of every period
        StorList[t][v]=StorList[t-1][v]-DemandList[v][t]+ProdList[t][v]
      BestTransPlan=BestTrans(StorList[t],Donors,Recievers,Sectors,nbSectors)
      TransList[t]=BestTransPlan
      for x1 in range(nbSectors):  #p1 gives to p2
        for x2 in range(nbSectors):
          StorList[t][x1]-=TransList[t][x1][x2]
          StorList[t][x2]+=TransList[t][x1][x2]
      for i in range(nbSectors):
          CurrentCost += Sectors[i].cp * ProdList[t][i] + Sectors[
                  i].cl * abs(StorList[t][i]) * Sectors[i].checkShort(StorList[t][i]) + Sectors[
                      i].cs * abs(StorList[t][i]) * Sectors[i].checkStock(StorList[t][i])
          for j in range(nbSectors):
            if (i != j):
                CurrentCost += Sectors[i].getQij(
                          Sectors[j]) * (TransList[t][i][j]//Sectors[i].v[j])
    if CurrentCost<MinCostProd :
      MinCostProd=CurrentCost
      BestProdPlan=[ProdList,StorList,TransList,MinCostProd]
  if notFirstTime : 
    test = True
    BestSol2=Bestmaintenance(BestProdPlan[0],Sectors,Mp,Mc,alpha,beta,lambda0,dt)
    Reservoire=InventoryMain(BestProdPlan[0],BestProdPlan[1],sigma,Sectors,Mp1,Mc1,alpha,beta,lambda0,dt,Qmax)
    print(Reservoire)
    CurrentCost=BestProdPlan[-1]
    CurrentTime = time.time()
    inbetween = CurrentTime-BaseTime
    PWS.write(row,1,inbetween)
    PWS.write(row,0,k)
    PWS.write(row,2,MinCostProd)
    row+=1
    for i in range(nbSectors):
        test = test==(BestSol2[i][1]==BestSol[i][1])
        CurrentCost += BestSol2[i][0]
    History[0].append(inbetween)
    History[1].append(i)
    History[2].append(CurrentCost)
    History[3].append(0)
    hum=Bestmaintenance(BestProdPlan[0],Sectors,Mp,Mc,alpha,beta,lambda0,dt)
    Reservoire=InventoryMain(BestProdPlan[0],BestProdPlan[1],sigma,Sectors,Mp,Mc,alpha,beta,lambda0,dt,Qmax)
    ExtraInf[-1]=hum
    History[4].append(sum([i[0] for i in hum]))
    try :
      for si in range(nbSectors):
        tempora=0
        for ti in range(nbSectors):
          tempora+= DemandList[si][ti]-StorList[ti][si]+90*invgauss.pdf(DemandList[si][ti], mu)
          for tr in range(nbSectors):
            tempora+=TransList[t][tr][si]-TransList[t][si][tr]
        if tempora>ProdList[ti][si]:
          Validate=False
    except : 
      print()
       
    if (CurrentCost<Mincost1 and test and validate):
      Mincost1=CurrentCost
      savedalt.append([CurrentCost, BestProdPlan,[test,BestSol,BestSol2]])
      np.save(f"/content/gdrive/MyDrive/QualitaResults/HistoryConverge{id}",savedalt)
      #BestResultWriter(param,BestProdPlan[0],BestProdPlan[1],BestProdPlan[2],DemandList,id)
      print(f"newCost with test {CurrentCost}")
      print(f"testingfunction cost : {ProdCost(BestProdPlan[0],BestProdPlan[1],BestProdPlan[2],BestSol2,param)}")
      CurrentTime = time.time()
      inbetween = CurrentTime-BaseTime
      print(inbetween)
      PWS.write(row,1,inbetween)
      PWS.write(row,0,k)
      PWS.write(row,2,MinCostProd)
      row+=1
      ManufacturingData=[BestProdPlan[0],BestProdPlan[1],BestProdPlan[2],DemandList]
      BestResultWriterEver(param,ExtraInf,ManufacturingData,History,id)
    if (CurrentCost < Mincost):
      saved.append([CurrentCost, BestProdPlan,[test,BestSol,BestSol2]])
      Mincost=CurrentCost
      np.save(f"/content/gdrive/MyDrive/QualitaResults/HistoryOverall{id}",saved)
      print(f"newCost with overall {CurrentCost}")
  BestSol = Bestmaintenance(BestProdPlan[0],Sectors,Mp,Mc,alpha,beta,lambda0,dt)
  notFirstTime = True